In [7]:
# Import packages
%reset -f
from helpers import load_all_data, vectorized_flatten, sigmoid, get_log_loss, get_accuracy, sigmoid_derivative, gradient_update, get_loss_plot, plot_loss
import numpy as np
import cv2

In [8]:
# Load and prepare data
def prep_data(data_path):
    '''
    --------------------
    Perceptron algorithm
    --------------------
    Parameters: 
    X: Numpy array of training features (shape = 784 X n)
    y: Binary (1/0) training label (shape = n X 1)
    --------------------
    Output: 
    
    w: trained weights
    y_hat: predictions
    --------------------
    '''
    # Get datasets
    X_train, y_train, X_dev, y_dev, X_test, y_test = load_all_data(data_path)
    
    # Flatten datasets
    X_train_flattened = vectorized_flatten(X_train)
    X_dev_flattened = vectorized_flatten(X_dev)
    X_test_flattened = vectorized_flatten(X_test)
    
    # Recode 0 to -1
    y_train[np.where(y_train == 0)] = -1
    y_test[np.where(y_train == 0)] = -1
    y_dev[np.where(y_train == 0)] = -1
    

    # Add extra column to Y_train
    y_train = y_train.reshape(1, -1)
    y_dev = y_dev.reshape(1, -1)
    y_test = y_test.reshape(1, -1)
    
    return(X_train_flattened, X_dev_flattened, X_test_flattened, y_train, y_dev, y_test)

In [9]:
def train_perceptron(X, Y, epochs):
    '''
    --------------------
    Perceptron algorithm
    --------------------
    Parameters: 
    X: Numpy array of training features (shape = 784 X n)
    y: Binary (1/-1) training label (shape = n X 1)
    --------------------
    Output: 
    w: trained weights
    b: trained biases
    y_preds: predictions 
    --------------------
    '''
    # Initialize weights and biases
    w = np.zeros(X.shape[0])
    b = 0
    
    # Run for a fixed number of epochs
    for epoch in range(1, epochs + 1): 
        
        # Do this for each example in the dataset
        for i in range(X.shape[1]):
            
            # Store the sample data
            x_i = X[:, i]
            y_i = Y[0][i]
            
            # Compute the prediction with the current weights
            if (np.dot(w, x_i) + b > 0): y_hat = 1
            else: y_hat = -1
            
            # Check if the prediction is correct against the labels
            # If it is correct we don't need to make any updates: we just move to the next iteration
            # If it is not correct then we do the following: 
            # 1) Update the error flag to show there is still a misclassification 
            # 2) Update the weights and biases in the direction of the label
            if y_hat != y_i:
                w = w + y_i*x_i
                b = b + y_i
            
            # Monitor accuracy every 6000 samples
            if i % 6000 == 0:
                
                # Get predictions
                y_preds = np.array([int(np.dot(w, X[:, i]) + b  > 0) for i in range(X.shape[1])])
        
                # Training accuracy                       
                train_accuracy = get_accuracy(Y, y_preds)
        
                # Get training accuracy
                print("Epoch {}/{}: Training_accuracy = {}".format(epoch, epochs, train_accuracy))
    
    # Return statement
    return(w, b)

In [10]:
def get_perceptron_performance(w, b, X, Y): 
    '''
    --------------------
    Run perceptron algorithm to get a base-line
    --------------------
    Parameters: 
    X: Numpy array of training features (shape = 784 X n)
    y: Binary (1/0) training label (shape = n X 1)
    --------------------
    Output: 
    w: trained weights
    y_preds: predictions
    --------------------
    '''
    # Get predictions
    y_preds = np.array([int(np.dot(w, X[:, i]) + b  > 0) for i in range(X.shape[1])])
        
    # Training accuracy                       
    train_accuracy = get_accuracy(Y, y_preds)
    
    # Return statement
    return(y_preds, train_accuracy)

In [19]:
def get_perceptron_baseline(data_path, epochs):
    '''
    --------------------
    Run perceptron algorithm to get a base-line
    --------------------
    Parameters: 
    X: Numpy array of training features (shape = 784 X n)
    y: Binary (1/0) training label (shape = n X 1)
    --------------------
    Output: 
    w: trained weights
    y_preds: predictions
    --------------------
    '''
    # Set the random seed for np.random number generator
    # This will make sure results are reproducible
    np.random.seed(132089)
    
    # Prepare data for the perceptron
    X_train_flattened, X_dev_flattened, X_test_flattened, y_train, y_dev, y_test = prep_data(data_path)
    
    # Call the perceptron training with the given epochs
    w, b = train_perceptron(X_train_flattened, y_train, epochs)
    
    # Get train set performance
    train_preds, train_accuracy = get_perceptron_performance(w, b, X_train_flattened, y_train)
    
    # Get dev set performance
    dev_preds, dev_accuracy = get_perceptron_performance(w, b, X_dev_flattened, y_dev)
    
    # Return statement
    return(w, b, train_preds, dev_preds, train_accuracy, dev_accuracy)

In [20]:
# Set parameters
data_path = '../setup/data'
epochs = 100

# Call training function
w, b, train_preds, dev_preds, train_accuracy, dev_accuracy = get_perceptron_baseline(data_path, epochs)

Epoch 1/100: Training_accuracy = 0.0
Epoch 1/100: Training_accuracy = 0.4275833333333333
Epoch 2/100: Training_accuracy = 0.49175
Epoch 2/100: Training_accuracy = 0.44175
Epoch 3/100: Training_accuracy = 0.49433333333333335
Epoch 3/100: Training_accuracy = 0.45975
Epoch 4/100: Training_accuracy = 0.488
Epoch 4/100: Training_accuracy = 0.379
Epoch 5/100: Training_accuracy = 0.4900833333333333
Epoch 5/100: Training_accuracy = 0.45416666666666666
Epoch 6/100: Training_accuracy = 0.4865
Epoch 6/100: Training_accuracy = 0.46908333333333335
Epoch 7/100: Training_accuracy = 0.4890833333333333
Epoch 7/100: Training_accuracy = 0.46408333333333335
Epoch 8/100: Training_accuracy = 0.493
Epoch 8/100: Training_accuracy = 0.46558333333333335
Epoch 9/100: Training_accuracy = 0.4925
Epoch 9/100: Training_accuracy = 0.46491666666666664
Epoch 10/100: Training_accuracy = 0.49333333333333335
Epoch 10/100: Training_accuracy = 0.45116666666666666
Epoch 11/100: Training_accuracy = 0.4905
Epoch 11/100: Traini

Epoch 85/100: Training_accuracy = 0.4940833333333333
Epoch 85/100: Training_accuracy = 0.47758333333333336
Epoch 86/100: Training_accuracy = 0.4945
Epoch 86/100: Training_accuracy = 0.47891666666666666
Epoch 87/100: Training_accuracy = 0.49391666666666667
Epoch 87/100: Training_accuracy = 0.4851666666666667
Epoch 88/100: Training_accuracy = 0.49533333333333335
Epoch 88/100: Training_accuracy = 0.48391666666666666
Epoch 89/100: Training_accuracy = 0.488
Epoch 89/100: Training_accuracy = 0.474
Epoch 90/100: Training_accuracy = 0.48891666666666667
Epoch 90/100: Training_accuracy = 0.47858333333333336
Epoch 91/100: Training_accuracy = 0.49616666666666664
Epoch 91/100: Training_accuracy = 0.4845
Epoch 92/100: Training_accuracy = 0.49566666666666664
Epoch 92/100: Training_accuracy = 0.48341666666666666
Epoch 93/100: Training_accuracy = 0.49216666666666664
Epoch 93/100: Training_accuracy = 0.4075
Epoch 94/100: Training_accuracy = 0.4960833333333333
Epoch 94/100: Training_accuracy = 0.43233333

In [21]:
# Show results
train_accuracy, dev_accuracy

(0.4945, 0.95)